# 14. 아이유 팬이 좋아할 만한 다른 아티스트 찾기

머신러닝 애플리케이션 중 가장 상업적인 성공을 거둔 추천 시스템에 대해 알아보자. 추천 시스템의 원리를 한 문장으로 줄인다면 나와 비슷한 다른 사용자들이 좋아하는 것과 비슷한 것을 내게 추천해준다. 어떻게 비슷한지 알 수 있을까? 그리고 상품이나 정보가 서로 유사한지는 어떻게 알 수 있을까? 추천 시스템의 기본 원리를 파악하고 만들어보자.

# 14-2. 추천 시스템이란 게 뭔가요?

온라인 콘텐츠 서비스에서 데이터 분석 + AI 기술을 접복한 추천 시스템은 선택이 아닌 필수가 됐다. 음원 서비스를 예로 들면 유튜브 뮤직, 스포티파이, 애플 뮤직 등 추천 시스템을 통한 개인화 서비스가 없는 곳을 찾기 힘들어졌다. 음원 차트로 유명한 멜론도 2020년 업데이트 하면서 개인 맞춤 서비스를 메인에 배치했다.   
음원 서비스 제공자들은 수많은 유저들이 어떤 아티스트의 노래를 들었는지 광범위한 데이터를 축적하고 있다. 이런 빅데이터가 정확한 추천의 원동력이 되고 있다. 그렇다면 추천의 원리가 어떤건지 구체적으로 알아보자. [콘텐츠 추천 알고리즘의 진화 기사](http://www.kocca.kr/insight/vol05/vol05_04.pdf)   

기사를 확인해 보면 협업 필터링과 콘텐츠 기반 필터링이 나온다. 협업 필터링은 사용자간의 유사성 혹은 아이템간의 유사성을 파악해서 추천해주는 것이고, 콘텐츠 기반 필터링은 아이템 자체를 분석하여 추천을 구현하는것이다.   

협업 필터링은 해당 자료의 데이터가 쌓일때까지 추천하기 어려운 콜드 스타트, 사용자 수가 많을수록 계산량 증가로 효율 하락, 대다수의 사람들이 인기있는 항목을 좋아하고 나머지 항목들이 추천을 받지 못하는 롱테일 문제가 있다. 그리고 콘텐츠 기반 필터링은 고전적 추천 알고리즘으로 콜드 스타트를 해결할 수 있지만, 사진과 비디오를 동시에 추천해야 하는 경우 항목에서 얻을 수 있는 정보가 달라서 추천 기능을 구현하기 어렵다.

__유저가 좋아하는 특정 아티스트와 유사한 다른 아티스트를 추천하는 추천 시스템을 만들어 보자.__ [Last.fm](https://www.last.fm/)에서 어떤 유저가 특정 아티스트의 노래를 몇 번 들었는지 [데이터](http://ocelma.net/MusicRecommendationDataset/lastfm-360K.html)를 제공하고 있다. 국내는 카카오에서 학술용으로 공개한 [멜론 데이터](https://arena.kakao.com/c/7)를 활용하면 관련 추천을 할 수 있게 된다.   

```
$ mkdir -p ~/aiffel/recommendata_iu/data/lastfm-dataset-360K
$ ln -s ~/data/lastfm-dataset-360K/* ~/aiffel/recommendata_iu/data/lastfm-dataset-360K
```   

# 14-3. 데이터 탐색하기와 전처리

데이터 준비하기. 오늘 활용할 데이터는 `tsv`파일이다.   
```
$ more ~/aiffel/recommendata_iu/data/lastfm-dataset-360K/usersha1-artmbid-artname-plays.tsv
```   

항목의 정의가 `user-mboxsha1 \t musicbrainz-artist-id \t artist-name \t plays`로 되어있는걸 봐서 user ID, artist MBID, artist name, play회수로 나눠져 있는걸 확인할 수 있다.

In [1]:
import pandas as pd
import os

fname = os.getenv('HOME') + '/aiffel/recommendata_iu/data/lastfm-dataset-360K/usersha1-artmbid-artname-plays.tsv'
col_names = ['user_id', 'artist_MBID', 'artist', 'play']   # 임의로 지정한 컬럼명
data = pd.read_csv(fname, sep='\t', names= col_names)      # sep='\t'로 주어야 tsv를 열 수 있습니다.  
data.head(10)

,user_id,artist_MBID,artist,play
0,00000c289a1829a808ac09c00daf10bc3c4e223b,3bd73256-3905-4f3a-97e2-8b341527f805,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,f2fb0ff0-5679-42ec-a55c-15109ce6e320,die Ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,b3ae82c2-e60b-4551-a76d-6620f1b456aa,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,3d6bbeb7-f90e-4d10-b440-e153c0d10b53,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,bbd2ffd7-17f4-4506-8572-c1ea58c3f9a8,juliette & the licks,706
5,00000c289a1829a808ac09c00daf10bc3c4e223b,8bfac288-ccc5-448d-9573-c33ea2aa5c30,red hot chili peppers,691
6,00000c289a1829a808ac09c00daf10bc3c4e223b,6531c8b1-76ea-4141-b270-eb1ac5b41375,magica,545
7,00000c289a1829a808ac09c00daf10bc3c4e223b,21f3573f-10cf-44b3-aeaa-26cccd8448b5,the black dahlia murder,507
8,00000c289a1829a808ac09c00daf10bc3c4e223b,c5db90c4-580d-4f33-b364-fbaa5a3a58b5,the murmurs,424
9,00000c289a1829a808ac09c00daf10bc3c4e223b,0639533a-0402-40ba-b6e0-18b067198b73,lunachicks,403


In [2]:
# 사용하는 컬럼만 남겨줍니다.
using_cols = ['user_id', 'artist', 'play']
data = data[using_cols]
data.head(10)

,user_id,artist,play
0,00000c289a1829a808ac09c00daf10bc3c4e223b,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,die Ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,juliette & the licks,706
5,00000c289a1829a808ac09c00daf10bc3c4e223b,red hot chili peppers,691
6,00000c289a1829a808ac09c00daf10bc3c4e223b,magica,545
7,00000c289a1829a808ac09c00daf10bc3c4e223b,the black dahlia murder,507
8,00000c289a1829a808ac09c00daf10bc3c4e223b,the murmurs,424
9,00000c289a1829a808ac09c00daf10bc3c4e223b,lunachicks,403


In [3]:
data['artist'] = data['artist'].str.lower() # 검색을 쉽게 하기 위해 아티스트 문자열을 소문자로 바꿔줍시다.
data.head(10)

,user_id,artist,play
0,00000c289a1829a808ac09c00daf10bc3c4e223b,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,die ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,juliette & the licks,706
5,00000c289a1829a808ac09c00daf10bc3c4e223b,red hot chili peppers,691
6,00000c289a1829a808ac09c00daf10bc3c4e223b,magica,545
7,00000c289a1829a808ac09c00daf10bc3c4e223b,the black dahlia murder,507
8,00000c289a1829a808ac09c00daf10bc3c4e223b,the murmurs,424
9,00000c289a1829a808ac09c00daf10bc3c4e223b,lunachicks,403


In [4]:
# 첫 번째 유저의 아티스트 목록 확인하기
condition = (data['user_id']== data.loc[0, 'user_id'])
data.loc[condition]

,user_id,artist,play
0,00000c289a1829a808ac09c00daf10bc3c4e223b,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,die ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,juliette & the licks,706
5,00000c289a1829a808ac09c00daf10bc3c4e223b,red hot chili peppers,691
6,00000c289a1829a808ac09c00daf10bc3c4e223b,magica,545
7,00000c289a1829a808ac09c00daf10bc3c4e223b,the black dahlia murder,507
8,00000c289a1829a808ac09c00daf10bc3c4e223b,the murmurs,424
9,00000c289a1829a808ac09c00daf10bc3c4e223b,lunachicks,403


__데이터 탐색__   

- 유저 수, 아티스트 수, 인기 많은 아티스트
- 유저들이 몇 명의 아티스트를 듣고 있는지 통계
- 유저 play 횟수 중앙값에 대한 통계

위의 항목들을 확인해보자. 그리고 `pandas.DataFrame.nunique()`는 특정 컬럼에 포함된 유니크한 데이터의 개수를 알아보는데 유용하다.

In [5]:
# 유저 수
data['user_id'].nunique()

358868

In [6]:
# 아티스트 수
data['artist'].nunique()

291346

In [7]:
# 인기 많은 아티스트
artist_count = data.groupby('artist')['user_id'].count()
artist_count.sort_values(ascending=False).head(30)

artist
radiohead                77254
the beatles              76245
coldplay                 66658
red hot chili peppers    48924
muse                     46954
metallica                45233
pink floyd               44443
the killers              41229
linkin park              39773
nirvana                  39479
system of a down         37267
queen                    34174
u2                       33206
daft punk                33001
the cure                 32624
led zeppelin             32295
placebo                  32072
depeche mode             31916
david bowie              31862
bob dylan                31799
death cab for cutie      31482
arctic monkeys           30348
foo fighters             30144
air                      29795
the rolling stones       29754
nine inch nails          28946
sigur rós                28901
green day                28732
massive attack           28691
moby                     28232
Name: user_id, dtype: int64

In [8]:
# 유저별 몇 명의 아티스트를 듣고 있는지에 대한 통계
user_count = data.groupby('user_id')['artist'].count()
user_count.describe()

count    358868.000000
mean         48.863234
std           8.524272
min           1.000000
25%          46.000000
50%          49.000000
75%          51.000000
max         166.000000
Name: artist, dtype: float64

In [9]:
# 유저별 play횟수 중앙값에 대한 통계
user_median = data.groupby('user_id')['play'].median()
user_median.describe()

count    358868.000000
mean        142.187676
std         213.089902
min           1.000000
25%          32.000000
50%          83.000000
75%         180.000000
max       50142.000000
Name: play, dtype: float64

__모델 검증을 위한 사용자 초기 정보 세팅__   
음악 취향과 비슷한 아티스트를 추천받기 위해 본인에 대한 정보를 입력해야한다.

In [10]:
# 본인이 좋아하시는 아티스트 데이터로 바꿔서 추가하셔도 됩니다! 단, 이름은 꼭 데이터셋에 있는 것과 동일하게 맞춰주세요. 
my_favorite = ['black eyed peas' , 'maroon5' ,'jason mraz' ,'coldplay' ,'beyoncé']

# 'zimin'이라는 user_id가 위 아티스트의 노래를 30회씩 들었다고 가정하겠습니다.
my_playlist = pd.DataFrame({'user_id': ['zimin']*5, 'artist': my_favorite, 'play':[30]*5})

if not data.isin({'user_id':['zimin']})['user_id'].any():  # user_id에 'zimin'이라는 데이터가 없다면
    data = data.append(my_playlist)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

data.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,user_id,artist,play
17535650,"sep 20, 2008",turbostaat,12
17535651,"sep 20, 2008",cuba missouri,11
17535652,"sep 20, 2008",little man tate,11
17535653,"sep 20, 2008",sigur rós,10
17535654,"sep 20, 2008",the smiths,10
0,zimin,black eyed peas,30
1,zimin,maroon5,30
2,zimin,jason mraz,30
3,zimin,coldplay,30
4,zimin,beyoncé,30


__모델에 활용하기 위한 전처리__   
우리가 다루는 데이터에 user와 artist에 번호를 붙이는 __indexing__ 작업을 하자.

In [11]:
# 고유한 유저, 아티스트를 찾아내는 코드
user_unique = data['user_id'].unique()
artist_unique = data['artist'].unique()

# 유저, 아티스트 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
artist_to_idx = {v:k for k,v in enumerate(artist_unique)}

In [12]:
# 인덱싱이 잘 되었는지 확인해 봅니다. 
print(user_to_idx['zimin'])    # 358869명의 유저 중 마지막으로 추가된 유저이니 358868이 나와야 합니다. 
print(artist_to_idx['black eyed peas'])

358868
376


In [13]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(data):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    data['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# artist_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_artist_data = data['artist'].map(artist_to_idx.get).dropna()
if len(temp_artist_data) == len(data):
    print('artist column indexing OK!!')
    data['artist'] = temp_artist_data
else:
    print('artist column indexing Fail!!')

data

user_id column indexing OK!!
artist column indexing OK!!


,user_id,artist,play
0,0,0,2137
1,0,1,1099
2,0,2,897
3,0,3,717
4,0,4,706
...,...,...,...
0,358868,376,30
1,358868,270115,30
2,358868,3746,30
3,358868,62,30


# 14-4. 사용자의 명시적 / 암묵적 평가

사용자들이 해당 항목을 얼마나 선호하는지 체크하기 위해 좋아요나 별점같은 명시적으로 나타내면 좋지만, 서비스를 사용하면서 자연스럽게 발생하는 암묵적인 피드백도 아이템에 대한 평가를 알 수 있는 단서가 된다. [Collabrative Filtering for Implicit Feedback Datasets](http://yifanhu.net/PUB/cf.pdf) 논문에서 암묵적 피드백 데이터셋을 활용할 때의 고민이 잘 담겨있다. 암묵적 피드백 데이터셋의 특징을 다음과 같이 정리하고 있다.   
- 부정적인 피드백이 없다. No Negative Feedback
- 잡음이 많다. Inherently Noisy
- 수치는 신뢰도를 의미한다. The numerical value of implicit feedback indicates confidence
- 암묵적 피드백 시스템의 평가는 적절한 방법을 고민해봐야 한다. Evaluation of implicit-feedback recommender requires appropriate measures

In [14]:
# 1회만 play한 데이터의 비율을 보는 코드
only_one = data[data['play']<2]
one, all_data = len(only_one), len(data)
print(f'{one},{all_data}')
print(f'Ratio of only_one over all data is {one/all_data:.2%}')  # f-format에 대한 설명은 https://bit.ly/2DTLqYU

147740,17535660
Ratio of only_one over all data is 0.84%


만약 아티스트의 곡을 한번만 들은 유저는 해당 아티스트를 좋아할까? 싫어할까?   
경우의 수가 많아 명확한 정답이 있지 않다. 이러한 암묵적 데이터들은 도메인 지식과 직관이 활용되어야한다. 그래서 다음과 같은 규칙을 적용해보자.   
- 한 번이라도 들었으면 선호한다고 판단한다.
- 많이 재생한 아티스트에 대해 가중치를 주어 더 확실히 좋아한다고 판단한다.

# 14-5. Matrix Factorization(MF)

m명의 사용자들이 n명의 아티스트를 평가한 데이터를 `m, n` 사이즈의 rating matrix를 만들어보자. 행렬 중 일부는 데이터가 비어 있다. 추천 시스템의 협업 필터링은 결국 이런 평가 행렬을 전제로 하는 것이다. 만약 평가 행렬의 비어 있는 부분을 포함한 완벽한 정보를 얻을 수 있다면 완벽한 추천이 가능해진다.   

추천 시스템의 다양한 모델 중 `Matrix Factorization(MF, 행렬 분해)` 모델을 사용하자. 이 모델은 2006년 Netflix에서 백만 달러의 상금을 걸고 개최한 자사 추천 시스템의 성능을 10% 이상 향상시키는 챌린지를 계기로 알려지게 되었다.   
![](https://d3s0tskafalll9.cloudfront.net/media/images/E-3v2-2_ekCv9hW.max-800x600.png)

(m,n) 사이즈의 행렬 R을 (m,k) 사이즈의 행렬 P와 (k,n) 사이즈의 행렬 Q로 분해할 수 있다면, R은 P, Q의 행렬곱으로 표현할 수 있다는 아이디어이다. k는 m이나 n보다 작은 값으로 계산량이 줄어 유리해진다. 아이디어는 단순하지만 MF 모델은 성능이 준수하고 Scalability(확장성)가 좋아 많이 사용된다.   

![](https://d3s0tskafalll9.cloudfront.net/media/images/E-3v2-3.max-800x600.png)

MF 모델을 사용자에게 영화 추천하는 모델에 대입해서 그렸다. 위 행렬 방식으로 말하면 m=4, n=5, k=2인 MF 모델이 된다.   

(m, k) 사이즈의 Feature Matrix P는 k차원의 벡터를 사용자 수 만큼 모은 행렬이다. 빨간색 모자를 쓰고 있는 사람의 첫 번째 벡터 $P_0=(1, 0.1)$은 __사용자의 feature 벡터__ 가 된다. Q 행렬의 첫 번째 벡터 $Q_0=(0.9, -0.2)$는 해리포터 __영화의 feature 벡터__ 가 된다.   
사용자와 영화의 벡터를 내적해서 얻어지는 0.88이 $R_{0,0}$으로 정의되는 사용자의 영화 선호도가 된다.   

모델의 목표는 모든 유저와 아이템에 대해 k-dimension의 벡터를 잘 만드는 것이다. 잘 만드는 기준은 유저 i의 벡터 $U_i$와 아이템 j의 벡터 $I_j$를 내적했을 때 유저가 아이템에 평가한 수치 $M_{ij}$와 비슷한지를 파악하면 된다.   
$U_i * I_j = M{ij}$

위의 collaborative filltering for implicit feedback datsets 논문에서 제안한 모델을 사용할 것이다.   

위에서 검증을 위한 사용자 초기 세팅에 'black eyed peas'를 play한 데이터를 추가했는데 사용자의 벡터와 'black eyed peas'의 벡터를 곱했을 때 1에 가까워야 모델이 잘 학습하는 것을 목표로 한다. 5번 들었어도 두 벡터를 곱했을 때 5 대신 1에 가까워져야 한다. 그러면 queen의 벡터를 곱해도 수치를 예상할 수 있게 된다.   

목표가 유저의 재생 횟수를 맞춰야 하는 것이라면 그에 맞는 다른 모델을 사용해야한다. 이는 [MF reconnender system](https://towardsdatascience.com/recommendation-system-matrix-factorization-d61978660b4b)을 확인해보자.

# 14-6. CSR(Compressed Sparse Row) Matrix

유저 X 아이템 평가 행렬을 생각해봤을 때 유저는 36만명, 아티스트는 29만명이다. 행렬로 표현하고 각 원소에 정수 한 개(1byte)가 들어간다면. 360,000 * 290,000 * 1byte = 97GB가 필요하게 된다. 이는 평가행렬의 대부분의 공간이 0으로 채워지고, 이런 행렬을 Sparse Matrix라고 한다. 그래서 메모리 낭비를 최소화하기 위해 유저가 들어본 아티스트의 정보만 저장하고 전체 행렬 형태를 유추할 수 있는 데이터 구조가 필요하다.   

그래서 CSR(Compressed Sparse Row) Matrix가 대안됐다. 0이 아닌 유효한 데이터로 채워지는 데이터의 값과 좌표 정보로 구성하여 메모리 사용량을 최소화하고 sparse한 matrix와 동일한 행렬을 표현할 수 있도록 하는 데이터 구조이다.   

CSR matrix는 data, indices, indptr로 행렬을 압축하여 표현한다.   
- data는 0이 아닌 원소를 차례로 기입한 값이다.
- indices는 data의 요소가 어느 열에 있는지 표현한 index이다. 
- indptr은 각 행에서 0이 아닌 첫 번째 원소가 data리스트에서 몇 번째에 해당하는지와, 마지막 data 벡터의 길이를 추가한 값이다. 이를 통해 data의 요소들이 어느 행에 있는지 알 수 있게 된다. 만약 0이 아닌 원소가 없는 경우, 그 다음 행의 값과 같은 값을 넣는다.   

![](https://d3s0tskafalll9.cloudfront.net/media/images/Screen_Shot_2022-02-10_at_4.29.43_PM.max-800x600.png)   
- data = [1,2,3,4,5,6] 
- indices = [0,4,1,3,0,3] 
- indptr = [0,2,4,4,6]

CSR matrix를 만드는 방법은 다양한데, [scipy.sparse.csr_matrix](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html) 링크를 통해 확인할 수 있다. 현재 data 구조에서는 `csr_matrix((data, (row_ind, col_ind)), [shape=(M, N)])
` 방식으로 만들어보자.

In [15]:
# 실습 위에 설명보고 이해해서 만들어보기
from scipy.sparse import csr_matrix

num_user = data['user_id'].nunique()
num_artist = data['artist'].nunique()

csr_data = csr_matrix((data.play, (data.user_id, data.artist)), shape= (num_user, num_artist))
csr_data

<358869x291347 sparse matrix of type '<class 'numpy.int64'>'
	with 17535578 stored elements in Compressed Sparse Row format>

# 14-7. MF 모델 학습하기

Matrix Factorization 모델을 [implicit](https://github.com/benfred/implicit) 패키지를 사용해 학습해보자.   
implicit 패키지는 암묵적(implicit) dataset을 사용한 다양한 모델을 굉장히 빠르게 학습할 수 있는 패키지이다. 이 패키지의 `als(AlternatingLeastSquares)` 모델을 사용해보자. Matrix Factorization에서 쪼개진 두 Feature Matrix를 한꺼번에 훈련하면 잘 수렴하지 않기 때문에, 한쪽을 고정하고 다른 쪽을 학습하는 방식을 번갈아 수행하는 als 방식이 효과적으로 알려져 있다.

In [16]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

`AlternatingLeastSquares` 클래스의 `__init__` 파라미터를 살펴보면 다음과 같다.   
- factors: 유저와 아이템의 벡터를 몇 차원으로 할 것인지
- regularization: 과적합을 방지하기 위해 정규화 값을 얼마나 사용할 것인지
- use_gpu: gpu 사용할 것인지
- iterations: epochs와 같은 의미로, 데이터를 몇 번 반복해서 학습할 것인지   

factors와 iterations를 늘릴수록 잘 학습하지만 over fitting의 우려가 있다.

In [17]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [18]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<291347x358869 sparse matrix of type '<class 'numpy.int64'>'
	with 17535578 stored elements in Compressed Sparse Column format>

In [19]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

2가지 사항을 살펴봐야한다.   
1. Zimin 벡터와 black eyed peas의 벡터를 어떻게 만들고 있는지
2. 두 벡터를 곱하면 어떤 값이 나오는지

In [20]:
zimin, black_eyed_peas = user_to_idx['zimin'], artist_to_idx['black eyed peas']
zimin_vector, black_eyed_peas_vector = als_model.user_factors[zimin], als_model.item_factors[black_eyed_peas]

print('슝=3')

슝=3


In [21]:
zimin_vector

array([ 0.69019634,  1.1206071 , -0.6394497 ,  0.13490348,  0.9468041 ,
       -0.19613029, -0.99977434,  0.1784035 ,  1.3646821 ,  0.02661193,
        0.7812069 ,  0.34483254, -0.14895731,  0.32586068,  1.359859  ,
       -0.6436787 ,  0.3590141 ,  0.31360725, -0.68865454, -0.2969451 ,
       -0.6309313 , -0.34995583,  0.3933402 ,  0.4151421 , -2.1349115 ,
       -0.78759325, -0.01872665, -0.60063404, -0.4040347 ,  0.4608407 ,
        0.3095348 ,  0.28032336,  0.14983939, -0.2559854 , -0.18029273,
        0.3144507 ,  1.3443887 , -0.6341968 ,  0.15445343, -0.69000447,
        0.30300683,  0.05794944, -0.11743802,  0.27390903, -0.6182563 ,
        0.75295305, -1.1179519 ,  0.8818909 , -0.28952742,  1.9356626 ,
        1.3017808 ,  0.04117525,  0.17808037,  0.23906873,  0.14216006,
       -0.5395452 ,  0.2852452 , -0.3491058 ,  0.09625563,  0.16431281,
        1.057246  ,  0.5313162 ,  0.00635241, -1.1524132 , -0.00884317,
        0.6344568 , -0.0064341 , -0.75537276, -0.10478093, -0.58

In [22]:
black_eyed_peas_vector

array([ 0.01632557,  0.02039555, -0.00293602,  0.00746351,  0.00446035,
        0.00872409, -0.01848995,  0.0130861 ,  0.01824906,  0.00989768,
        0.01681684,  0.01665859,  0.00415828,  0.00193829,  0.01888192,
        0.00147463,  0.0098205 ,  0.01149671, -0.00053328, -0.00689826,
        0.00219237,  0.0103724 ,  0.01578214,  0.0133338 , -0.02173982,
       -0.00693119,  0.00743185,  0.01569415,  0.00881545,  0.0132991 ,
        0.01096225,  0.02088743,  0.01067787,  0.00444954, -0.01549591,
        0.01241761,  0.019093  ,  0.00418954,  0.01473251, -0.01525253,
        0.0071879 ,  0.00322269,  0.00412722,  0.00325651, -0.00028835,
        0.01193106, -0.00975228,  0.01437715, -0.00504235,  0.0358281 ,
        0.01967715,  0.01415129,  0.0162615 ,  0.02891916,  0.02550719,
       -0.00210219,  0.01042794, -0.00080365, -0.01009732,  0.0048676 ,
        0.02281268,  0.00961776, -0.0047611 ,  0.00141758,  0.00879837,
       -0.00494563,  0.01214829,  0.00116034,  0.02452267,  0.00

In [23]:
# zimin과 black_eyed_peas를 내적하는 코드
np.dot(zimin_vector, black_eyed_peas_vector)

0.522833

1이 나와야 하는데 한참 낮은 0.49가 나왔다. factors를 늘리거나 iterations를 늘려야한다. 그리고 queen에 대한 선호도도 예측해서 확인해보자.

In [24]:
queen = artist_to_idx['queen']
queen_vector = als_model.item_factors[queen]
np.dot(zimin_vector, queen_vector)

0.31879956

0.29가 나왔는데 이 결과만 갖고 모델의 선호 기준을 정하기 어렵다. 왜냐하면 모델 만드는 사람이 정하기 나름이기 때문이다. 객관적 지표로 만들어 기준을 정하거나, 도메인 경험을 통해 정할 수도 있다. 추천 시스템은 다른 머신러닝 task보다 객관적인 평가가 어려운 분야이다.

# 14-8. 비슷한 아티스트 찾기 + 유저에게 추천하기

`AlternatingLeastSquares` 클래스에 구현되어 있는 `similar_items` 메서드를 통해 비슷한 아티스트를 찾아보자. 처음에는 `coldplay`로 찾아보자.

In [25]:
favorite_artist = 'coldplay'
artist_id = artist_to_idx[favorite_artist]
similar_artist = als_model.similar_items(artist_id, N=15)
similar_artist

[(62, 1.0000001),
 (277, 0.99058366),
 (28, 0.9820916),
 (5, 0.981234),
 (473, 0.97942686),
 (217, 0.9792431),
 (247, 0.97525704),
 (490, 0.97118056),
 (910, 0.9644829),
 (1018, 0.9614307),
 (418, 0.9587778),
 (694, 0.9562881),
 (268, 0.9502147),
 (782, 0.9491846),
 (55, 0.9430464)]

결과를 보면 (아티스트 id, 유사도) tuple 형태로 반환하고 있다. 아티스트의 id를 아티스트의 이름으로 매핑해보자.

In [26]:
#artist_to_idx 를 뒤집어, index로부터 artist 이름을 얻는 dict를 생성합니다. 
idx_to_artist = {v:k for k,v in artist_to_idx.items()}
[idx_to_artist[i[0]] for i in similar_artist]

['coldplay',
 'muse',
 'the killers',
 'red hot chili peppers',
 'placebo',
 'radiohead',
 'the beatles',
 'oasis',
 'nirvana',
 'the smashing pumpkins',
 'u2',
 'foo fighters',
 'pink floyd',
 'the white stripes',
 'arctic monkeys']

In [27]:
def get_similar_artist(artist_name: str):
    artist_id = artist_to_idx[artist_name]
    similar_artist = als_model.similar_items(artist_id)
    similar_artist = [idx_to_artist[i[0]] for i in similar_artist]
    return similar_artist

print("슝=3")

슝=3


In [28]:
get_similar_artist('2pac')

['2pac',
 'notorious b.i.g.',
 'nas',
 'snoop dogg',
 'the game',
 'dr. dre',
 'jay-z',
 '50 cent',
 'ice cube',
 'common']

In [29]:
get_similar_artist('lady gaga')

['lady gaga',
 'rihanna',
 'britney spears',
 'katy perry',
 'beyoncé',
 'the pussycat dolls',
 'christina aguilera',
 'leona lewis',
 'kelly clarkson',
 'gwen stefani']

특정 장르의 마니아들은 해당 장르만 선호도가 집중되고 특성이 두드러진다. 

__유저에게 아티스트 추천하기__   

`AlternatingLeastSquares` 클래스에 구현되어 있는 `recommed` 메서드를 통해 좋아할만한 아티스트를 추천받자. `filter_already_liked_items`는 유저가 이미 평가한 아이템은 제외하는 Argument이다.

In [30]:
user = user_to_idx['zimin']
# recommend에서는 user*item CSR Matrix를 받습니다.
artist_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
artist_recommended

[(350, 0.4684322),
 (369, 0.454418),
 (550, 0.4541876),
 (1800, 0.4454369),
 (724, 0.42897123),
 (2249, 0.427438),
 (409, 0.42425418),
 (274, 0.41385263),
 (627, 0.40445453),
 (354, 0.40437543),
 (901, 0.40011054),
 (355, 0.39294407),
 (564, 0.38952935),
 (391, 0.37692058),
 (24, 0.37681478),
 (618, 0.37365952),
 (382, 0.3716715),
 (5555, 0.37078458),
 (944, 0.36590478),
 (621, 0.36342078)]

In [31]:
[idx_to_artist[i[0]] for i in artist_recommended]

['rihanna',
 'justin timberlake',
 'britney spears',
 'lady gaga',
 'lily allen',
 'katy perry',
 'amy winehouse',
 'michael jackson',
 'maroon 5',
 'nelly furtado',
 'pink',
 'madonna',
 'kanye west',
 'christina aguilera',
 'jack johnson',
 'the pussycat dolls',
 'mika',
 'timbaland',
 'avril lavigne',
 'alicia keys']

rihanna를 추천해주고 있는데, 왜 그럴까? `explain` 메서드를 사용하면 기록을 남긴 데이터 중 __추천에 기여한 정도__ 를 확인할 수 있다.

In [32]:
rihanna = artist_to_idx['rihanna']
explain = als_model.explain(user, csr_data, itemid=rihanna)

In [33]:
[(idx_to_artist[i[0]], i[1]) for i in explain[1]]

[('beyoncé', 0.2233897249849475),
 ('black eyed peas', 0.1510942784778954),
 ('jason mraz', 0.05572448087533553),
 ('coldplay', 0.03961286372694918),
 ('maroon5', 6.884397382317091e-05)]

추천 시스템에서 Baseline으로 많이 사용되는 MF를 통해 유저에게 아티스트 추천 task를 만들어 봤다. 하지만 baseline이기 때문에 __아쉬운 점__도 있다.   

유저, 아티스트에 대한 Meta 정보를 반영하기 쉽지 않다. 예를들어 연령대별로 음악 취향이 굉장히 다를 수 있다. 그리고 유저가 언제 play 했는지 반영하기 쉽지 않다. 10년 전 즐겨듣던 아티스트와 지금 즐겨듣는 아티스트가 다를 수 있다.   

이러한 이유와 딥러닝의 발전으로 MF 외에 다양한 모델 구조도 많이 연구하고 사용되고 있다. 하지만 핵심은 MF와 비슷하다. 유저와 아이템에 대한 벡터를 잘 학습하여 취향에 맞게 아이템을 보여주거나(retrieval) 걸러내는(filtering) 역할을 한다.

# 14-9. 프로젝트 

MF 모델 학습 방법을 토대로 영화 추천 시스템을 제작해보자. 데이터셋은 추천 시스템의 MNIST라고 부를만한 Movielens이다.   

유저가 영화에 대해 평점을 매긴 데이터가 크기별로 있는데 이번에는 `MovieLens 1M Dataset`을 사용한다. 별점 데이터는 대표적인 명시적(explicit) 평가 데이터인데, 암묵적(implicit) 평가 데이터로 간주하고 테스트해보자. 어떻게 하냐면 __별점을 시청횟수__로 해석하자. __3점 미만__으로 준 데이터는 __선호하지 않는다__고 가정하자.

```
$ mkdir -p ~/aiffel/recommendata_iu/data/ml-1m
$ ln -s ~/data/ml-1m/* ~/aiffel/recommendata_iu/data/ml-1m
```